In [2]:
import xlrd
import json
# from google.colab import files
# uploaded = files.upload()

In [3]:
book = xlrd.open_workbook("RTW.xls")
# print(json.dumps(final_result,indent=1))


In [4]:
print("The number of worksheets is {0}".format(book.nsheets))
print("Worksheet name(s): {0}".format(book.sheet_names()))

The number of worksheets is 19
Worksheet name(s): ['JOHOR', 'MELAKA', 'N9', 'ZON SELATAN', 'PERLIS', 'KEDAH', 'PP', 'PERAK', 'ZON UTARA', 'PAHANG', 'KELANTAN', 'TERENGGANU', 'ZON TIMUR', 'SABAH', 'SARAWAK', 'ZON BORNEO', 'KUALA LUMPUR', 'SELANGOR', 'ZON LEMBAH KLANG NEW']


In [5]:
worksheet = book.sheet_by_index(0)
print("-----------------\n")
print("Service Type\n") 
print("-----------------\n")
serv_type = []
for a in worksheet.row_values(0):
  if(a!=""):
    serv_type.append(a)
print(json.dumps(serv_type,indent=1))

-----------------

Service Type

-----------------

[
 "PHYSIOTHERAPY",
 "OCCUPATIONAL THERAPY",
 "SPEECH\nTHERAPY",
 "PSYCHOLOGY",
 "CONSULTATION",
 "ASSESSMENT & REPORT",
 "FUNCTIONAL CAPACITY EVALUATION & REPORT",
 "ADMINISTRATION SERVICES",
 "HOSTEL",
 "DRY NEEDLING/\nACUPUNCTURE",
 "SERVICES BY MODALITIES",
 "SERVICES BY PACKAGE"
]


<h3> How Many Services Does a Service Type Has ? i.e., COLUMN COUNT FOR EACH SERVICE</h3>

In [6]:
print("-----------------\n")
print("SERV___COLUMNS\n") 
print("-----------------\n")
serv_columns = {}
col = [24,18,1,1,4,14,10,2,1,1,62,14]
for i in range(0,len(serv_type)):
   serv_columns.update({serv_type[i]:col[i]})
print(json.dumps(serv_columns,indent=1))

-----------------

SERV___COLUMNS

-----------------

{
 "PHYSIOTHERAPY": 24,
 "OCCUPATIONAL THERAPY": 18,
 "SPEECH\nTHERAPY": 1,
 "PSYCHOLOGY": 1,
 "CONSULTATION": 4,
 "ASSESSMENT & REPORT": 14,
 "FUNCTIONAL CAPACITY EVALUATION & REPORT": 10,
 "ADMINISTRATION SERVICES": 2,
 "HOSTEL": 1,
 "DRY NEEDLING/\nACUPUNCTURE": 1,
 "SERVICES BY MODALITIES": 62,
 "SERVICES BY PACKAGE": 14
}


<h1> Services</h1>

In [7]:
print("-----------------\n")
print("Service\n") 
print("-----------------\n")
serv = []
for a in worksheet.row_values(1):
  if(a!=""):
    serv.append(a.strip("\n"))

# Removing Bil & Panel Treatment options
serv.pop(0)
serv.pop(0)
print(json.dumps(serv,indent=1))


-----------------

Service

-----------------

[
 " Physiotherapy \nTreatment",
 "Manual Therapy",
 "Pain Management",
 "Musculoskeletal Rehabilitation",
 "Sport Injuries \nRehabilitation",
 "Stroke rehabilitation",
 "Spinal Cord Injury Rehabilitation",
 "Traumatic Brain Injury Rehabilitation",
 "Amputee \nRehabilitation",
 "Cardiorespiratory Rehabilitation",
 "Hydrotherapy",
 "Home Therapy + Mileage",
 "Group Therapy And Exercises",
 "Burn Management",
 "Patient And Family Education Including Talk Workshop And Training",
 "Women And Men Health",
 "Other Neurological Or Rare Diseases Rehabilitation Such As Parkinson/Multiple Sclerosis/Muscular Dysthropy/Systemic Lupus/Ankylosing Spondylotis/Gullain Barre 1Syndrome",
 "Alter-G (Anti Graviti) Physio ",
 "Tbion (Tibion Bionic Technology) Rehab",
 "Chest Physiotherapy Manual",
 "Chest Physiotherapy Suction Machine",
 "Hand Function Training",
 "Fine Motor Training",
 "Cognitive Training",
 "Occupational Therapy Treatment",
 "Adl Training (

<h1>Mapping Hierarchy :-</h1>
<h2>Panel & Service TO Service Type TO State Mapping  </h2>

<h3>Price to Service Mapping </h3>

In [8]:
d = []
jsonfile = open("json_dump_new.txt","w+")

In [20]:

def price(state_name,service_type,service,j,lower_bound,worksheet_no):
  c = {}
  worksheet = book.sheet_by_index(worksheet_no)
  for i in range(2,lower_bound):
    a = worksheet.cell_value(rowx=i,colx=j)
    b = worksheet.cell_value(rowx=i,colx=1)
    c.update({state_name:{service_type:{service:{b:a}}}})
    return c
  #   jsonfile.write(",".join(json.dumps(c,indent=1)))
  #   print(c)
  #   d.append(c)
  # print("-----------------------------\n")
  # print(d)
  # print("-----------------------------\n")
  # # jsonfile.write(",".join(d))
  # return d

<h3> Service To Service Type Mapping<h3>

In [22]:
def service_to_service_type(state_name,lower_bound,worksheet_no):
  panel_to_serv_price = {}
  serv_to_serv_price = {}
  panel_to_serv = []
  k = 2
  for key,value in serv_columns.items():
    for i in range(k,k+value):
      # panel_to_serv.update({key:{serv[i-2]: price(i,lower_bound,worksheet_no)}})
      panel_to_serv.append(price(state_name,key,serv[i-2],i,lower_bound,worksheet_no))
      # panel_to_serv_price.update({serv[i-2]: price(state_name,key,serv[i-2],i,lower_bound,worksheet_no)})
    # serv_to_serv_price.update({key:panel_to_serv_price})
    k = k + value
  return panel_to_serv
  # print("*******************************************\n")
  # print(json.dumps(panel_to_serv,indent=1))
  # print("*******************************************\n")
  # return panel_to_serv
 

<h3>State To Service Type Mapping </h3>

In [23]:
print("------------------------------")
print("Final Data For Each Sheet")
print("------------------------------")
a = 0
final_result = []
sheet_column_count = {}
state_to_service = {}
for i in range(0,19):
    for j in (book.sheet_by_index(i)).col_values(1):
        if(j!=""):
            a+=1
    sheet_column_count.update({book.sheet_by_index(i).name:a-2})
    final_result.append(service_to_service_type(book.sheet_by_index(i).name,a-2,i))
    # state_to_service.update({book.sheet_by_index(i).name:service_to_service_type(a-2,i)})
    a=0
jsonfile.close()
file1 = open("json_dump1.txt","w+")
print(json.dumps(final_result,indent=1))
file1.write(json.dumps(final_result, indent=1)) 

------------------------------
Final Data For Each Sheet
------------------------------
[
 [
  {
   "JOHOR": {
    "PHYSIOTHERAPY": {
     " Physiotherapy \nTreatment": {
      "SINAR REHABILITATION CENTRE (SEGAMAT)": 80.0
     }
    }
   }
  },
  {
   "JOHOR": {
    "PHYSIOTHERAPY": {
     "Manual Therapy": {
      "SINAR REHABILITATION CENTRE (SEGAMAT)": ""
     }
    }
   }
  },
  {
   "JOHOR": {
    "PHYSIOTHERAPY": {
     "Pain Management": {
      "SINAR REHABILITATION CENTRE (SEGAMAT)": ""
     }
    }
   }
  },
  {
   "JOHOR": {
    "PHYSIOTHERAPY": {
     "Musculoskeletal Rehabilitation": {
      "SINAR REHABILITATION CENTRE (SEGAMAT)": ""
     }
    }
   }
  },
  {
   "JOHOR": {
    "PHYSIOTHERAPY": {
     "Sport Injuries \nRehabilitation": {
      "SINAR REHABILITATION CENTRE (SEGAMAT)": ""
     }
    }
   }
  },
  {
   "JOHOR": {
    "PHYSIOTHERAPY": {
     "Stroke rehabilitation": {
      "SINAR REHABILITATION CENTRE (SEGAMAT)": ""
     }
    }
   }
  },
  {
   "JOHOR": {


470600